# Library

In [1]:
import sys
import os
directory_path = os.path.abspath(os.path.join('..'))
utils_path = os.path.abspath(os.path.join('../utils'))
if directory_path not in sys.path:
    sys.path.append(directory_path)
    sys.path.append(utils_path)

In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import warnings
from pyvis.network import Network as net
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings("ignore")
from utils.Validator import *
from utils.Combinations import *
from utils.NetworkDraw import *
pd.set_option('display.max_columns', None)

# Network Attribute

## Followers

In [3]:
df_GUF_attr = pd.read_csv(r"../data/processed/df_GUF_scaled.csv")
df_GUF_attr["UserId"] = df_GUF_attr["UserId"].astype(str)
df_GUF_attr.head()

,UserId,UserName,in_degree_followers,out_degree_followers,in_degree_cent_followers,pagerank_cent_followers,out_degree_followers_scaled,in_degree_cent_followers_scaled,pagerank_cent_followers_scaled
0,993,benhamner,1513,20,0.005959,0.002447,20.580552,27.187648,25.229307
1,368,antgoldbloom,983,6,0.003872,0.001775,20.174165,24.669834,23.791618
2,1902,solorzano,207,32,0.000815,0.000135,20.928882,20.983373,20.285634
3,1950,sskiing,29,26,0.000114,0.000072,20.754717,20.137767,20.152212
4,3258,NaN,0,22,0.000000,0.000001,20.638607,20.000000,20.000000


In [4]:
df_GUF_attr.shape

(253893, 9)

In [17]:
GUF = nx.read_gexf(r"../data/processed/GUF.gexf")
print(nx.info(GUF))

Name: 
Type: DiGraph
Number of nodes: 253893
Number of edges: 765508
Average in degree:   3.0151
Average out degree:   3.0151


In [22]:
#label
dict_tmp = dict(df_GUF_attr[["UserId","UserName"]].values.tolist())
nx.set_node_attributes(GUF, name='label', values=dict_tmp)

#Tamanio
dict_values = dict(df_GUF_attr[["UserId","in_degree_cent_followers_scaled"]].values.tolist())
nx.set_node_attributes(GUF, name='size', values=dict_values)

#Labels
dict_ig = dict(df_GUF_attr[["UserId","in_degree_followers"]].values.tolist())
dict_og = dict(df_GUF_attr[["UserId","out_degree_followers"]].values.tolist())
dict_pr = dict(df_GUF_attr[["UserId","pagerank_cent_followers"]].values.tolist())

list_title = []
for n in GUF.nodes:
    #Mto Recibido y Enviado
    ig = dict_ig[n]
    formatted_ig = "{:,.0f}".format(ig)
    og = dict_og[n]
    formatted_og = "{:,.0f}".format(og)
    pr = dict_pr[n]
    formatted_pr = "{:,.4f}".format(pr)
    # Creamos el html final
    title_text = " Seguidores: "+str(formatted_ig) + "\n Sigue a: "+str(formatted_og)+ "\n Pagerank: "+str(formatted_pr)
    list_title.append((n,title_text))
dict_title = dict(list_title)
nx.set_node_attributes(GUF, name='title', values=dict_title)

In [34]:
nx.write_gexf(GUF, r"../data/processed/GUF_attr.gexf")

## Teams

In [19]:
df_GUT_attr = pd.read_csv(r"../data/processed/df_GUT_scaled.csv")
df_GUT_attr["UserId"] = df_GUT_attr["UserId"].astype(str)
df_GUT_attr.tail()

,UserId,UserName,degree_teams,degree_cent_teams,eigenvector_cent_teams,degree_teams_scaled,degree_cent_teams_scaled,eigenvector_cent_teams_scaled
155725,7479659,NaN,1,0.000006,5.427340e-23,20.000000,20.000000,20.0
155726,9998652,NaN,1,0.000006,5.427340e-23,20.000000,20.000000,20.0
155727,6066760,NaN,1,0.000006,5.427340e-23,20.000000,20.000000,20.0
155728,99987,chrismoulder,4,0.000026,5.427340e-23,21.818182,21.818182,20.0
155729,999897,NaN,1,0.000006,5.427340e-23,20.000000,20.000000,20.0


In [20]:
print(df_GUT_attr.shape)

(155730, 8)


In [21]:
df_GUT_attr.loc[df_GUT_attr["UserName"].isnull(), 'UserName'] = "Usuario NC"
print(df_GUT_attr.shape)

(155730, 8)


In [22]:
GUT = nx.read_gexf(r"../data/processed/GUT.gexf")
print(nx.info(GUT))

Name: 
Type: Graph
Number of nodes: 155730
Number of edges: 196736
Average degree:   2.5266


In [23]:
#label
dict_tmp = dict(df_GUT_attr[["UserId","UserName"]].values.tolist())
nx.set_node_attributes(GUT, name='label', values=dict_tmp)

#Tamanio
dict_values = dict(df_GUT_attr[["UserId","degree_cent_teams_scaled"]].values.tolist())
nx.set_node_attributes(GUT, name='size', values=dict_values)

#Labels
dict_dg = dict(df_GUT_attr[["UserId","degree_teams"]].values.tolist())
dict_ev = dict(df_GUT_attr[["UserId","eigenvector_cent_teams"]].values.tolist())

list_title = []
for n in GUT.nodes:
    #Mto Recibido y Enviado
    dg = dict_dg[n]
    formatted_dg = "{:,.0f}".format(dg)
    ev = dict_ev[n]
    formatted_ev = "{:,.4f}".format(ev)
    # Creamos el html final
    title_text = " Grado: "+str(formatted_dg) + "\n Eigenvector: "+str(formatted_ev)
    list_title.append((n,title_text))
dict_title = dict(list_title)
nx.set_node_attributes(GUT, name='title', values=dict_title)

In [24]:
nx.write_gexf(GUT, r"../data/processed/GUT_attr.gexf")

## Forum

In [33]:
df_GUM_attr = pd.read_csv(r"../data/processed/df_GUM_scaled.csv")
df_GUM_attr["UserId"] = df_GUM_attr["UserId"].astype(str)
df_GUM_attr.tail()

,UserId,UserName,degree_forums,degree_cent_forums,eigenvector_cent_forums,degree_cent_forums_scaled,eigenvector_cent_forums_scaled
101279,997470,captivus,3,0.000030,2.265569e-06,20.005122,20.000477
101280,99791,shimaa,4,0.000039,1.988875e-07,20.010243,20.000042
101281,999082,lukewiebolt,10,0.000099,1.421804e-04,20.040973,20.029906
101282,999317,bednarb,3,0.000030,1.551812e-08,20.005122,20.000003
101283,99954,leoaguirre,6,0.000059,1.590878e-08,20.020487,20.000003


In [34]:
print(df_GUM_attr.shape)

(101284, 7)


In [35]:
df_GUM_attr.loc[df_GUM_attr["UserName"].isnull(), 'UserName'] = "Usuario NC"
print(df_GUM_attr.shape)

(101284, 7)


In [36]:
GUM = nx.read_gexf(r"../data/processed/GUM.gexf")
print(nx.info(GUM))

Name: 
Type: Graph
Number of nodes: 101284
Number of edges: 4306750
Average degree:  85.0430


In [37]:
#label
dict_tmp = dict(df_GUM_attr[["UserId","UserName"]].values.tolist())
nx.set_node_attributes(GUM, name='label', values=dict_tmp)

#Tamanio
dict_values = dict(df_GUM_attr[["UserId","degree_cent_forums_scaled"]].values.tolist())
nx.set_node_attributes(GUM, name='size', values=dict_values)

#Labels
dict_dg = dict(df_GUM_attr[["UserId","degree_forums"]].values.tolist())
dict_ev = dict(df_GUM_attr[["UserId","eigenvector_cent_forums"]].values.tolist())

list_title = []
for n in GUM.nodes:
    #Mto Recibido y Enviado
    dg = dict_dg[n]
    formatted_dg = "{:,.0f}".format(dg)
    ev = dict_ev[n]
    formatted_ev = "{:,.4f}".format(ev)
    # Creamos el html final
    title_text = " Grado: "+str(formatted_dg) + "\n Eigenvector: "+str(formatted_ev)
    list_title.append((n,title_text))
dict_title = dict(list_title)
nx.set_node_attributes(GUM, name='title', values=dict_title)

In [44]:
nx.write_gexf(GUM, r"../data/processed/GUM_attr.gexf")

# Function

In [3]:
def get_followers_graph(UserId,_with_second_level=False):
    GUF = nx.read_gexf(r"../data/processed/GUF_attr.gexf")
    SG = NetworkDraw.get_subgraph_from_with_neighbors(GUF,UserId,with_second_level=_with_second_level)
    #Asignamos el atributo de color
    predecessors = list(GUF.predecessors(UserId))
    successors = list(GUF.successors(UserId))
    list_color = []
    for n in GUF.nodes:
        var = (0,0)
        if n == UserId:
            var = (n,"#FA5748") # NODO DE CONSULTA
        elif n in predecessors or n in successors: #2DO NIVEL
            var = (n,"#61C0FA")
        else:
            var = (n,"#4D82E3") #3ER NIVEL
        list_color.append(var)
    dict_color = dict(list_color)
    nx.set_node_attributes(GUF, name='color', values=dict_color)
    #Dibujamos el grafo
    NetworkDraw.draw_graph3(SG,notebook=False,directed=True,only_physics_buttons=False,height="100%",width="100%",
                            output_filename=r"../others/"+UserId+'_followers.html')

In [4]:
def get_teams_graph(UserId,_with_second_level=False):
    GUT = nx.read_gexf(r"../data/processed/GUT_attr.gexf")
    SG = NetworkDraw.get_subgraph_from_with_neighbors(GUT,UserId,with_second_level=_with_second_level,directed=False)
    #Asignamos el atributo de color
    neighbors = list(GUT.neighbors(UserId))
    list_color = []
    for n in GUT.nodes:
        var = (0,0)
        if n == UserId:
            var = (n,"#FA5748") # NODO DE CONSULTA
        elif n in neighbors: #2DO NIVEL
            var = (n,"#61C0FA")
        else:
            var = (n,"#4D82E3") #3ER NIVEL
        list_color.append(var)
    dict_color = dict(list_color)
    nx.set_node_attributes(GUT, name='color', values=dict_color)
    #Dibujamos el grafo
    NetworkDraw.draw_graph3(SG,notebook=False,directed=False,only_physics_buttons=False,height="100%",width="100%",
                            output_filename=r"../others/"+UserId+'_teams.html')

In [5]:
def get_forum_graph(UserId,_with_second_level=False):
    GUM = nx.read_gexf(r"../data/processed/GUM_attr.gexf")
    SG = NetworkDraw.get_subgraph_from_with_neighbors(GUM,UserId,with_second_level=_with_second_level,directed=False)
    #Asignamos el atributo de color
    neighbors = list(GUM.neighbors(UserId))
    list_color = []
    for n in GUM.nodes:
        var = (0,0)
        if n == UserId:
            var = (n,"#FA5748") # NODO DE CONSULTA
        elif n in neighbors: #2DO NIVEL
            var = (n,"#61C0FA")
        else:
            var = (n,"#4D82E3") #3ER NIVEL
        list_color.append(var)
    dict_color = dict(list_color)
    nx.set_node_attributes(GUM, name='color', values=dict_color)
    #Dibujamos el grafo
    NetworkDraw.draw_graph3(SG,notebook=False,directed=False,only_physics_buttons=False,height="100%",width="100%",
                            output_filename=r"../others/"+UserId+'_forum.html')

# Call Function

In [38]:
get_followers_graph(UserId = "4494269",_with_second_level=False)

In [6]:
get_teams_graph(UserId = "4494269",_with_second_level=False)

In [7]:
get_forum_graph(UserId = "4494269",_with_second_level=False)